In [1]:
# Phcli Jupyter Python Template
# 
# 使用手册：
# 1. 请将全局变量定义在第一个输入区内
# 2. Phcli 会自动在第二个输入区初始化 Spark Session
# 3. 所有 print 会在 phcli maxauto dag 后自动转为 logger.debug() 方法
# 4. 请在第三个输入区开始编码，phcli maxauto dag 后会全部归类为一个方法


# Config defined in here

############## == config == ###################
job_name = "compare_patient_with_detection"
job_runtime = "python3"
job_command = "submit"
job_timeout = 720.0
############## == config == ###################


# Variables defined in here

############## == input args == ###################
g_input_paramater = "Empty"
g_partition_num = 2
############## == input args == ###################

############## == output args == ###################
g_out_parameter = "Empty"
############## == output args == ###################

############## == preset function == ###################
from phcli.ph_max_auto.ph_hook.get_abs_path import get_result_path
from phcli.ph_max_auto.ph_hook.get_abs_path import get_depends_path
# result_path_prefix = get_result_path({"name":job_name})
# depends_path = get_depends_path({"name":job_name})
############## == preset function == ###################


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("yarn") \
    .appName("application name") \
    .config("spark.driver.cores", "1") \
    .config("spark.driver.memory", "1g") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.memory", "2g") \
    .config("spark.executor.instances", "2") \
    .config("spark.sql.execution.arrow.pyspark.enabled", True) \
    .getOrCreate()

In [3]:
from pyspark.sql.types import IntegerType, DoubleType, StringType, StructType
from pyspark.sql.functions import col, date_format, count, isnull, lit,first
from pyspark.sql.functions import when, isnan, pandas_udf, PandasUDFType, datediff
from pyspark.sql import functions as Func
from pyspark.sql import DataFrame
from pyspark.sql import Window
from typing import Iterator
import pandas as pd
import re


In [4]:
# %%

##
##

In [4]:
# %%
## ====== 输入文件和输出文件 ======

# 
g_whether_save_result = True

p_main_dir = "s3://ph-origin-files/user/zazhao/2020年结果-csv/"


# 输入目录
p_patient_union_drug_analyse_std = "s3://ph-origin-files/user/zazhao/2020年结果-csv/HIS_result/patient_union_drug_analyse_result/"
p_detection_std = "s3://ph-origin-files/user/zazhao/2020年结果-csv/HIS_result/clean_detection_result/"

# 输出文件
p_patient_std_compare_out = p_main_dir + "HIS_result/" + "compare_data_result_with_detection"
p_quinolone_result_dir = p_patient_std_compare_out + "/quinolone_result"
p_cephalosporin_result_dir =p_patient_std_compare_out + "/cephalosporin_result"

In [5]:
# %%

## 读取病人处方数据
df_patient_union_analyse_std = spark.read.parquet(p_patient_union_drug_analyse_std)


In [ ]:
# %%

## 得到门诊 处方数据
# df_patient_in_hospital =

In [7]:
df_patient_union_analyse_std.select("severe_case").distinct().show()
# show(1, vertical=True)

+-----------+
|severe_case|
+-----------+
|          Y|
|          N|
+-----------+



In [ ]:
# %%

## 得到住院 处方数据
df_patient_out_hospital = df_patient_union_analyse_std.filter(col("就诊类型")=="住院")
df_detection_cleaned =df_patient_out_hospital.groupBy("患者ID","就诊序号","就诊类型","标准性别","年龄区间", "标准处方日期", 
                                "标准诊断", "标准科室", "formula", "single_or_formula", "mole_comb", "金额"  )\
                        .agg( Func.last("severe_case").alias("severe_case"), 
                                 Func.first("std_charge_type").alias("std_charge_type") )
                        .withColumn("PROC", "RX")\
#                         .withColumn("RxID", "RX")    需要添加唯一 ID


df_hosp_time = df_patient_out_hospital.select("患者ID","就诊序号", "标准住院时间", "标准出院时间").distinct()
    
df_rx_time = df_detection_cleaned.select("患者ID","就诊序号", "RxID", "标准处方日期" )


#### 得到第一个处方的信息
df_patient_visit = df_detection_cleaned.withColumn("标准处方日期顺序", 
                                                   Func.row_number().over(  
                                                           Windows.partitionBy("患者ID","就诊序号").orderBy(col("标准处方日期").asc() ) ) )
df_patient_visit = df_patient_visit.filter( col("标准处方日期顺序")==1)  
df_patient_visit = df_patient_visit.select(  ["患者ID","就诊序号","就诊类型","心律不齐","其他心血管疾病",
                                        "脑血管疾病","神经系统疾病","高血糖","高血脂","肝功能异常","肾功能异常",
                                        "结缔组织病","COPD","哮喘","支气管扩张","恶性实体瘤","心衰","白细胞计数",
                                        "C反应蛋白","降钙素原","嗜肺军团菌",
                                        "肺炎衣原体","肺炎支原体","冠状病毒","合胞病毒","流感病毒","腺病毒",
                                        "柯萨奇病毒","鲍曼氏不动杆菌","大肠埃希菌","肺炎克雷伯菌","肺炎链球菌","金黄色葡萄球菌",
                                        "流感嗜血菌","嗜麦芽寡养单胞菌","嗜麦芽窄食单胞菌","铜绿假单胞菌","阴沟肠杆菌",
                                        "seg1_grp1","seg1_grp2","seg2_grp1","seg3_grp1","seg3_grp2","seg3_grp3"] ).distinct()



df_rx_list = df_detection_cleaned.selct("RxID", "formula", "single_or_formula", "mole_comb", "sales")

In [ ]:
# %%

## 读取检测数据
df_detection_std = spark.read.parquet(p_detection_std)
# 选择所有 病原菌检测  的类别
# df_detection_std = df_detection_std.filter( col("std_subject")=="病原菌检测")

df_detection_cleaned = df_detection_std.select('患者ID', '就诊序号', "REQUESTED_DATE_TIME_STD", "RESULTS_RPT_DATE_TIME_STD"
                                                  "pathogenic_type", "ABNORMAL_INDICATOR").distinc()


## 住院的检测信息清洗
df_detection_cleaned = df_detection_cleaned.join(df_hosp_time, on=["患者ID","就诊序号"] )
df_detection_cleaned = df_detection_cleaned.filter( col("pathogenic_type").isNotNull() )
df_detection_cleaned = df_detection_cleaned.withColumn("RESULTS_RPT_DATE_TIME_STD", 
                                                        Func.when( col("RESULTS_RPT_DATE_TIME_STD").isNull(),  
                                                                     Func.date_add( col("REQUESTED_DATE_TIME_STD"), 1) ) 
                                                        Func.when( ( col("RESULTS_RPT_DATE_TIME_STD").isNotNull() )& \
                                                                     ( Func.datediff( col("RESULTS_RPT_DATE_TIME"), col("标准出院时间"))>0 ),
                                                                      Func.date_add( col("REQUESTED_DATE_TIME_STD"), 1) )
                                                      )
#                                             .withColumn("ID")
########################################### 添加唯一的自增 ID


df_detection_cleaned = df_detection_cleaned.withColumnRenamed("RESULTS_RPT_DATE_TIME_STD", "R" )\
                                            .withColumnRenamed("REQUESTED_DATE_TIME_STD", "REQ" )



df_detection_cleaned_time = df_detection_cleaned.select('患者ID', '就诊序号', "ID", "R", "REQ")



df_detection_cleaned_rx_test = None



In [8]:
# df_detection_std.columns

In [7]:
# # %%

# ## 读取检测数据
# df_detection_std = spark.read.parquet(p_detection_std)
# # 选择所有 病原菌检测  的类别
# df_detection_std = df_detection_std.filter( col("std_subject")=="病原菌检测")



In [54]:
# %%


df_temp_detection = df_detection_std.select(["患者ID", "就诊序号", "检测日期" ] )\
                                        .withColumn("处方检测匹配日期",  date_format( col("检测日期"), "yyyyMMdd" ) )\
                                        .withColumn("检测日期",  date_format( col("检测日期"), "yyyyMMdd" ) )
                                        

# ## 因为每个病人一天会有多个检测,所以需要去重
df_temp_detection = df_temp_detection.distinct()
# 就诊序号有空值存在
df_temp_detection = df_temp_detection.where( col("就诊序号").isNotNull() )


df_temp_patient = df_patient_union_analyse_std.withColumn("处方检测匹配日期", date_format( col("处方日期"), "yyyyMMdd" ) )\
                                                .withColumn("标准处方日期_INT",  col("标准处方日期").cast("int"))

print( df_temp_detection.count()  )
print( df_temp_patient.join(df_temp_detection,  on= ["患者ID", "就诊序号", "处方检测匹配日期"], how="inner").count()  )

# 此处只考虑存在一次检测的情况下
# 区分检测前和 检测后
df_patient_with_detection = df_temp_patient.join(df_temp_detection,  on= ["患者ID", "就诊序号", "处方检测匹配日期"], how="inner") \
                                            .select( ["医院ID","患者ID","OUT_ID","就诊类型", "检测日期"]).distinct()



#### 判断是否存在多次检测
# df_patient_with_detection.select(["医院ID","患者ID","就诊序号", "OUT_ID","就诊类型", "标准入院时间", "标准出院时间", "标准处方日期", "检测日期"])\
#                             .dropDuplicates( ["医院ID","患者ID","OUT_ID","就诊类型", "检测日期"])\
#                             .withColumn( "多次检测",  Func.count("*").over( Window.partitionBy( "医院ID","患者ID","OUT_ID","就诊类型" ) ) )\
#                             .filter( col("多次检测")>1)\
#                             .show(10, False )
###


# 在 "医院ID","患者ID","OUT_ID","就诊类型" 层面下 出现多次检测日期
# 选择最小的检测日期
df_patient_with_detection = df_patient_with_detection.groupBy( "医院ID","患者ID","OUT_ID","就诊类型" ).agg(  
                                    Func.min( col("检测日期").cast("int") ).alias("MIN_检测日期_INT") )


df_patient_std = df_temp_patient.join( df_patient_with_detection, on=["医院ID","患者ID","OUT_ID","就诊类型"],how="left" )\
                                .withColumn("数据内容",  Func.when( ( col("MIN_检测日期_INT").isNotNull() ) & \
                                                                       ( ( col("标准处方日期_INT") - col("MIN_检测日期_INT"))<0  ), "检测前" )\
                                                              .when( ( col("MIN_检测日期_INT").isNotNull() ) & \
                                                                    (  ( col("标准处方日期_INT")- col("MIN_检测日期_INT") )>0 ), "检测后" )\
                                                              .when(   col("标准处方日期_INT") == col("MIN_检测日期_INT") , "检测当天" )\
                                                              .otherwise("未检测") )


print( df_patient_with_detection.count(), df_temp_patient.count(),  df_patient_std.count()  )

31692
22476
5271 461033 461033


In [70]:
# body_info = [["Bom", 20, 97.6, 165, "2015-04-08"],
#              ["Bom", 25, 97.6, 165, "2015-04-23"],
#              ["Alice", 23, 90.0, 160, "2015-04-09"],
#              ["kuke", 33, 190.0, 170, "2015-04-09"],
#              ["jike", 19, 120.0, 170,"2015-04-08"],
#              ["Joe", 24, 89.0, 162, "2015-04-31"]]

# body_df = spark.createDataFrame(body_info, ["name", "age", "weight", "height", "time"])
# body_df.show()


# # body_df.withColumn("aa", Func.row_number().over( Window.partitionBy("height").orderBy( col("age").asc() ) ) ).filter( col("aa")==1)\
# #         .show()

# body_df.withColumn("aa", Func.row_number().over( Window.partitionBy("height").orderBy( col("time").asc() ) ) )\
#         .show()

In [53]:
# %%

######## 此处的目的 是为了得到 uni_code 层面下,第一次处方  以便后面去匹配

## 采用这样的方式是为了得到 uni_code 层面下,第一次处方
# tag_win = Window.partitionBy("医院ID","患者ID","OUT_ID","就诊类型").orderBy( col("标准处方日期").asc() )
# df_patient_std = df_patient_std.withColumn("MIN_标准处方日期", Func.row_number().over(tag_win)) \
#                                .filter( col("MIN_标准处方日期") == 1 ) \
df_patient_std = df_patient_std.filter( col("SEQ")==1)  

df_patient_tag = df_patient_std.select(  ["医院ID","患者ID","OUT_ID","就诊类型","心律不齐","其他心血管疾病",
                                        "脑血管疾病","神经系统疾病","高血糖","高血脂","肝功能异常","肾功能异常",
                                        "结缔组织病","COPD","哮喘","支气管扩张","恶性实体瘤","心衰","白细胞计数",
                                        "C反应蛋白","降钙素原","嗜肺军团菌",
                                        "肺炎衣原体","肺炎支原体","冠状病毒","合胞病毒","流感病毒","腺病毒",
                                        "柯萨奇病毒","鲍曼氏不动杆菌","大肠埃希菌","肺炎克雷伯菌","肺炎链球菌","金黄色葡萄球菌",
                                        "流感嗜血菌","嗜麦芽寡养单胞菌","嗜麦芽窄食单胞菌","铜绿假单胞菌","阴沟肠杆菌",
                                        "seg1_grp1","seg1_grp2","seg2_grp1","seg3_grp1","seg3_grp2","seg3_grp3"] ).distinct()
df_patient_tag.count()

39865

In [55]:
# %%

df_table_4_m = df_patient_std.groupBy(["医院ID","患者ID","OUT_ID","就诊类型","标准医保类型","标准性别","年龄区间","标准诊断",
                                         "severe_case","标准科室","formula","mole_comb","single_or_formula","SEQ","标准处方日期", "数据内容"])\
                                .agg( Func.sum(col("金额")).alias("sales") ) \
                                .withColumn("treat_unicode",Func.format_string( "%s_%s_%s_%s", 
                                            col("医院ID"),col("患者ID"),col("OUT_ID"),col("就诊类型") ) )\
                                .withColumn("标准处方日期", col("标准处方日期").cast("int"))

#########################################################
#### 标准处方日期 是否需要 转换成 整数形式后才方便进行判断
#########################################################

In [58]:
# %%

# table_4_m初始喹诺酮类换药

## 选择第一次用药为  喹诺酮类 的患者   第一次处方记录
df_quinolone_before = df_table_4_m.filter( (col("formula").rlike("氟喹诺酮") )
                                                & (col("SEQ") == "1"))


## 找到 初次使用 喹诺酮类的患者, 并在随后 出现换药的患者记录
#1.先找到 初次使用喹诺酮类的患者  的 所有记录

df_quinolone_after = df_table_4_m.join( df_quinolone_before.select("treat_unicode").distinct(), on=["treat_unicode"], how="inner")

#2.筛选得到第一次换药的记录
win1 = Window.partitionBy("treat_unicode").orderBy( col("标准处方日期").asc() )
df_quinolone_after = df_quinolone_after.withColumn("first_formula",  Func.lit("氟喹诺酮"))\
                                    .filter( ~( col("formula") == col("first_formula")) )\
                                    .withColumn("换药处方次序",  Func.row_number().over(win1) )\
                                    .filter( col("换药处方次序")==1)

# df_quinolone_after = df_quinolone_after.dropDuplicates( ["treat_unicode"])                        


#3.筛除一直使用 喹诺酮类的患者 (及不存在换药的)
df_quinolone_before = df_quinolone_before.join(df_quinolone_after.select("treat_unicode"), on="treat_unicode", how="inner")

df_quinolone_before = df_quinolone_before.dropDuplicates( ["treat_unicode"])

# 将字段名换成对应的
df_quinolone_before = df_quinolone_before.withColumnRenamed("severe_case","severe_case_before") \
                                            .withColumnRenamed("formula","formula_before") \
                                            .withColumnRenamed("mole_comb","mole_comb_before") \
                                            .withColumnRenamed("single_or_formula","single_or_formula_before") \
                                            .withColumnRenamed("SEQ","SEQ_before") \
                                            .withColumnRenamed("标准处方日期","std_rx_date_before") \
                                            .withColumnRenamed("sales","sales_before")\
                                            .withColumnRenamed("数据内容","检测前处方")\

# 选择需要的 列
df_quinolone_before = df_quinolone_before.select("医院ID","患者ID","OUT_ID","就诊类型",
                                                 "severe_case_before","formula_before","mole_comb_before",
                                                 "single_or_formula_before","SEQ_before","std_rx_date_before",
                                                 "sales_before", "检测前处方")

df_quinolone_after = df_quinolone_after.withColumnRenamed("severe_case","severe_case_after")\
                                        .withColumnRenamed("formula","formula_after") \
                                        .withColumnRenamed("mole_comb","mole_comb_after") \
                                        .withColumnRenamed("single_or_formula","single_or_formula_after") \
                                        .withColumnRenamed("SEQ","SEQ_after") \
                                        .withColumnRenamed("标准处方日期","std_rx_date_after") \
                                        .withColumnRenamed("sales","sales_after")\
                                        .withColumnRenamed("数据内容","检测后处方")\


#4.合并换药前和换药的 标签信息
df_quinolone_delivery = df_quinolone_after.join(df_quinolone_before, on=["医院ID","患者ID","OUT_ID","就诊类型"], how="left")
print( df_quinolone_before.count(),  df_quinolone_after.count(), df_quinolone_delivery.count()  )

# 
df_quinolone_delivery = df_quinolone_delivery.join(df_patient_tag,["医院ID","患者ID","OUT_ID","就诊类型"],"left")
print( df_quinolone_delivery.count() )

# 判断诊断前后
df_quinolone_delivery = df_quinolone_delivery.withColumn("检测前后是否换药", Func.when( col("检测前处方")!=col("检测后处方"), 1).otherwise(0) )

# df_quinolone_delivery.show(1, vertical=True)

3737 3737 3737
3790
-RECORD 0----------------------------------------
 医院ID                   | SH0000005            
 患者ID                   | 11b65a478ec728816... 
 OUT_ID                   | 1                    
 就诊类型                 | 住院                 
 treat_unicode            | SH0000005_11b65a4... 
 标准医保类型             | 其他                 
 标准性别                 | 男                   
 年龄                     | 0                    
 标准诊断                 | 肺炎                 
 severe_case_after        | N                    
 标准科室                 | 儿科                 
 formula_after            | 青霉素类             
 mole_comb_after          | 哌拉西林,他唑巴坦    
 single_or_formula_after  | 单药                 
 SEQ_after                | 2                    
 std_rx_date_after        | 20200802             
 检测后处方               | 未检测               
 sales_after              | 209.64               
 first_formula            | 氟喹诺酮             
 换药处方次序             | 1                    

In [59]:
df_quinolone_delivery.filter( col("检测前后是否换药")==1).count()

701

In [62]:
# %%

#初始头孢类换药
df_cephalosporin_before = df_table_4_m.filter( (col("formula").rlike("头孢菌素类") )
                                                    & (col("SEQ") == "1"))

# 初次使用 头孢的患者  所有处方记录
df_cephalosporin_after = df_table_4_m.join( df_cephalosporin_before.select("treat_unicode").distinct(), on=["treat_unicode"], how="inner")
# 第一次换药的处方
win2 = Window.partitionBy("treat_unicode").orderBy( col("标准处方日期").asc() )
df_cephalosporin_after = df_cephalosporin_after.withColumn("first_formula",  Func.lit("头孢菌素类"))\
                                    .filter( ~( col("formula") == col("first_formula")) )\
                                    .withColumn("换药处方次序",  Func.row_number().over(win2) )\
                                    .filter( col("换药处方次序")==1)


# df_cephalosporin_after = df_cephalosporin_after.drop_duplicates(["treat_unicode"])

# 筛除不换药的患者，所有相关处方
df_cephalosporin_before = df_cephalosporin_before.join(df_cephalosporin_after.select("treat_unicode"), on="treat_unicode", how="inner")
df_cephalosporin_before = df_cephalosporin_before.drop_duplicates(["treat_unicode"])

# 
df_cephalosporin_before = df_cephalosporin_before.withColumnRenamed("severe_case","severe_case_before") \
                                                    .withColumnRenamed("formula","formula_before") \
                                                    .withColumnRenamed("mole_comb","mole_comb_before") \
                                                    .withColumnRenamed("single_or_formula","single_or_formula_before") \
                                                    .withColumnRenamed("SEQ","SEQ_before") \
                                                    .withColumnRenamed("标准处方日期","std_rx_date_before") \
                                                    .withColumnRenamed("sales","sales_before")\
                                                    .withColumnRenamed("数据内容","检测前处方")

df_cephalosporin_before = df_cephalosporin_before.select("医院ID","患者ID","OUT_ID","就诊类型",
                                                         "severe_case_before","formula_before","mole_comb_before",
                                                         "single_or_formula_before","SEQ_before","std_rx_date_before",
                                                         "sales_before", "检测前处方")

df_cephalosporin_after = df_cephalosporin_after.withColumnRenamed("severe_case","severe_case_after")\
                                                    .withColumnRenamed("formula","formula_after") \
                                                    .withColumnRenamed("mole_comb","mole_comb_after") \
                                                    .withColumnRenamed("single_or_formula","single_or_formula_after") \
                                                    .withColumnRenamed("SEQ","SEQ_after") \
                                                    .withColumnRenamed("标准处方日期","std_rx_date_after") \
                                                    .withColumnRenamed("sales","sales_after")\
                                                    .withColumnRenamed("数据内容","检测后处方")

# ************
df_cephalosporin_delivery = df_cephalosporin_after.join(df_cephalosporin_before,["医院ID","患者ID","OUT_ID","就诊类型"],"left")
print( df_cephalosporin_after.count(), df_cephalosporin_before.count(), df_cephalosporin_delivery.count() )

df_cephalosporin_delivery = df_cephalosporin_delivery.join(df_patient_tag,["医院ID","患者ID","OUT_ID","就诊类型"],"left")
print(df_cephalosporin_delivery.count())

df_cephalosporin_delivery = df_cephalosporin_delivery.withColumn("检测前后是否换药", Func.when( col("检测前处方")!=col("检测后处方"), 1).otherwise(0) )

2203 2203 2203
2223


In [63]:
df_cephalosporin_delivery.where( col("检测前后是否换药")==1).count()

503

In [35]:
# %%

# 进行判断重症医学科的操作
# df_quinolone_delivery = df_quinolone_delivery.withColumn("std_dept.x",when(df_quinolone_delivery["标准科室"] == "重症医学科",
#                                                                      "ICU").otherwise(col("标准诊断")))

# df_cephalosporin_delivery = df_cephalosporin_delivery.withColumn("std_dept.x",when(df_cephalosporin_delivery["标准科室"] == "重症医学科",
#                                                                              "ICU").otherwise(col("标准诊断")))

In [ ]:
# %%

## 判断检测前后是否换药

In [64]:
# %%

## 根据提交表格的需求选择 指定的列

columns_in_table = ["患者ID","就诊序号","就诊类型","标准医保类型","标准性别","年龄区间","标准诊断","标准科室",
                     "formula_before", "single_or_formula_before", "molecule_before", "sales_before", 
                     "formula_after", "single_or_formula_after", "molecule_after", "sales_after",
                     "检测前后是否换药", "嗜肺军团菌", "肺炎衣原体", "肺炎支原体", "冠状病毒", "合胞病毒", 
                     "流感病毒", "腺病毒", "柯萨奇病毒", "鲍曼氏不动杆菌", "大肠埃希菌", "肺炎克雷伯菌", 
                     "肺炎链球菌", "金黄色葡萄球菌", "流感嗜血菌", "嗜麦芽寡养单胞菌", "嗜麦芽窄食单胞菌", 
                     "铜绿假单胞菌", "阴沟肠杆菌"]

columns_need_in_result = ["PATIENT_ID", "VISIT_ID", "VISIT_TYPE", "std_charge_type", "GENDER", "age_range", "std_diag", "severe_case", "std_dept"
                            "formula_before", "single_or_formula_before", "molecule_before", "sales_before", 
                            "formula_after", "single_or_formula_after", "molecule_after", "sales_after",
                            "检测前后是否换药", "嗜肺军团菌", "肺炎衣原体", "肺炎支原体", "冠状病毒", "合胞病毒", 
                            "流感病毒", "腺病毒", "柯萨奇病毒", "鲍曼氏不动杆菌", "大肠埃希菌", "肺炎克雷伯菌", 
                            "肺炎链球菌", "金黄色葡萄球菌", "流感嗜血菌", "嗜麦芽寡养单胞菌", "嗜麦芽窄食单胞菌", 
                            "铜绿假单胞菌", "阴沟肠杆菌"]

df_result_quinolone = df_quinolone_delivery.select([ col(i).alias(j)  for i, j in zip(columns_in_table, columns_need_in_result)])

df_result_cephalosporin = df_cephalosporin_delivery.select([ col(i).alias(j)  for i, j in zip(columns_in_table, columns_need_in_result)])
# "PATIENT_ID", "VISIT_ID", "VISIT_TYPE", "std_charge_type", "GENDER", "age_range", "std_diag", "severe_case", "std_dept",

AnalysisException: cannot resolve '`就诊序号`' given input columns: [COPD, C反应蛋白, OUT_ID, SEQ_after, SEQ_before, first_formula, formula_after, formula_before, mole_comb_after, mole_comb_before, sales_after, sales_before, seg1_grp1, seg1_grp2, seg2_grp1, seg3_grp1, seg3_grp2, seg3_grp3, severe_case_after, severe_case_before, single_or_formula_after, single_or_formula_before, std_rx_date_after, std_rx_date_before, treat_unicode, 其他心血管疾病, 冠状病毒, 医院ID, 合胞病毒, 哮喘, 嗜肺军团菌, 嗜麦芽寡养单胞菌, 嗜麦芽窄食单胞菌, 大肠埃希菌, 就诊类型, 年龄, 心律不齐, 心衰, 恶性实体瘤, 患者ID, 换药处方次序, 支气管扩张, 柯萨奇病毒, 标准医保类型, 标准性别, 标准科室, 标准诊断, 检测前后是否换药, 检测前处方, 检测后处方, 流感嗜血菌, 流感病毒, 白细胞计数, 神经系统疾病, 结缔组织病, 肝功能异常, 肺炎克雷伯菌, 肺炎支原体, 肺炎衣原体, 肺炎链球菌, 肾功能异常, 脑血管疾病, 腺病毒, 金黄色葡萄球菌, 铜绿假单胞菌, 阴沟肠杆菌, 降钙素原, 高血糖, 高血脂, 鲍曼氏不动杆菌];;
'Project [患者ID#2 AS PATIENT_ID#18479, '就诊序号 AS VISIT_ID#18480, 就诊类型#1 AS VISIT_TYPE#18481, 标准医保类型#52 AS std_charge_type#18482, 标准性别#53 AS GENDER#18483, '年龄区间 AS age_range#18484, 标准诊断#32 AS std_diag#18485, 标准科室#51 AS severe_case#18486, formula_before#14906 AS std

In [19]:
# %%

df_quinolone_delivery.repartition(g_partition_num).write\
    .mode("overwrite").format("CSV").save( p_quinolone_result_dir)

df_cephalosporin_delivery.repartition(g_partition_num).write\
    .mode("overwrite").format("CSV").save(p_cephalosporin_result_dir)